In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, matthews_corrcoef, roc_auc_score, precision_recall_curve, auc
import os
import random
import gc

## old model

In [13]:
# from einops import rearrange, repeat
# import math


# class DSW_embedding(nn.Module):
#     def __init__(self, seg_len, d_model):
#         super(DSW_embedding, self).__init__()
#         self.seg_len = seg_len
#         self.d_model = d_model
#         self.linear = nn.Linear(seg_len * 231, d_model)

#     def forward(self, x):
#         batch, ts_len, ts_dim = x.shape
#         pad_len = (self.seg_len - (ts_len % self.seg_len)) % self.seg_len
#         if pad_len != 0:
#             x = torch.cat([x, torch.zeros(batch, pad_len, ts_dim).to(x.device)], dim=1)
#         seg_num = (ts_len + pad_len) // self.seg_len
#         x_segment = rearrange(x, 'b (seg_num seg_len) d -> b seg_num (seg_len d)', seg_len=self.seg_len)
#         x_embed = self.linear(x_segment)
#         # Reshape to (batch, seg_num * seg_len, d_model)
#         x_embed = x_embed.unsqueeze(2).repeat(1, 1, self.seg_len, 1)
#         x_embed = x_embed.view(batch, -1, self.d_model)
#         return x_embed[:, :ts_len, :]  # Remove the padding

# class SegMerging(nn.Module):
#     def __init__(self, d_model, win_size, norm_layer=nn.LayerNorm):
#         super().__init__()
#         self.d_model = d_model
#         self.win_size = win_size
#         self.linear_trans = nn.Linear(d_model, d_model)
#         self.norm = norm_layer(d_model)

#     def forward(self, x):
        
#         x = self.norm(x)
#         x = self.linear_trans(x)
#         return x

# class AttentionLayer(nn.Module):
#     def __init__(self, d_model, n_heads, dropout=0.1):
#         super(AttentionLayer, self).__init__()
#         self.n_heads = n_heads
#         self.d_model = d_model
#         self.d_k = d_model // n_heads

#         self.qkv_proj = nn.Linear(d_model, 3 * d_model)
#         self.o_proj = nn.Linear(d_model, d_model)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, q, k, v):
#         B, L, _ = q.size()
#         qkv = self.qkv_proj(q).chunk(3, dim=-1)
#         q, k, v = [rearrange(x, 'b l (h d) -> b h l d', h=self.n_heads) for x in qkv]

#         scores = torch.einsum('bhld,bhmd->bhlm', q, k) / math.sqrt(self.d_k)
#         attn = scores.softmax(dim=-1)
#         attn = self.dropout(attn)

#         context = torch.einsum('bhlm,bhmd->bhld', attn, v)
#         context = rearrange(context, 'b h l d -> b l (h d)')
#         output = self.o_proj(context)
#         return output

# class CrossEncoder(nn.Module):
#     def __init__(self, d_model, n_heads, d_ff, block_depth, dropout, factor, e_blocks, win_size):
#         super().__init__()
#         self.time_attention = AttentionLayer(d_model, n_heads, dropout)
#         self.dim_sender = AttentionLayer(d_model, n_heads, dropout)
#         self.dim_receiver = AttentionLayer(d_model, n_heads, dropout)
#         self.router = nn.Parameter(torch.randn(e_blocks, factor, d_model))
        
#         self.dropout = nn.Dropout(dropout)
#         self.norm1 = nn.LayerNorm(d_model)
#         self.norm2 = nn.LayerNorm(d_model)
#         self.norm3 = nn.LayerNorm(d_model)
#         self.norm4 = nn.LayerNorm(d_model)
        
#         self.MLP1 = nn.Sequential(nn.Linear(d_model, d_ff),
#                                   nn.GELU(),
#                                   nn.Linear(d_ff, d_model))
#         self.MLP2 = nn.Sequential(nn.Linear(d_model, d_ff),
#                                   nn.GELU(),
#                                   nn.Linear(d_ff, d_model))

#     def forward(self, x):
#         batch, seq_len, d_model = x.shape
#         time_enc = self.time_attention(x, x, x)
#         dim_in = x + self.dropout(time_enc)
#         dim_in = self.norm1(dim_in)
#         dim_in = dim_in + self.dropout(self.MLP1(dim_in))
#         dim_in = self.norm2(dim_in)

#         dim_send = dim_in
#         batch_router = repeat(self.router, 'e_blocks factor d_model -> (repeat e_blocks) factor d_model', repeat=batch)
#         dim_buffer = self.dim_sender(batch_router, dim_send, dim_send)
#         dim_receive = self.dim_receiver(dim_send, dim_buffer, dim_buffer)
#         dim_enc = dim_send + self.dropout(dim_receive)
#         dim_enc = self.norm3(dim_enc)
#         dim_enc = dim_enc + self.dropout(self.MLP2(dim_enc))
#         dim_enc = self.norm4(dim_enc)

#         return dim_enc

# class EncoderOnlyCrossFormer(nn.Module):
#     def __init__(self, seg_len, d_model, num_classes, num_layers, win_size, n_heads, d_ff, e_blocks, dropout=0.1):
#         super(EncoderOnlyCrossFormer, self).__init__()
        
#         # Embedding layer
#         self.embedding = DSW_embedding(seg_len, d_model)
        
#         # Encoder layers
#         self.encoders = nn.ModuleList([
#             CrossEncoder(d_model=d_model, n_heads=n_heads, d_ff=d_ff, block_depth=1, dropout=dropout, factor=5, e_blocks=e_blocks, win_size=win_size) for _ in range(num_layers)
#         ])
        
#         # Segment merging layer
#         self.seg_merge = SegMerging(d_model, win_size)
        
#         # Classification head
#         self.logit = nn.Linear(d_model, num_classes)
#         self.classifier = nn.Sequential(
#             nn.LayerNorm(d_model),
#             self.logit
#         )
        
#     def forward(self, x):
#         # Apply embedding
#         x_embed = self.embedding(x)
        
#         # Pass through each encoder layer
#         for encoder in self.encoders:
#             x_embed = encoder(x_embed)
        
#         # Merge segments
#         x_merged = self.seg_merge(x_embed)
        
#         # Classification head
#         logits = self.classifier(x_merged)
        
#         return logits

In [14]:
# model_1 = EncoderOnlyCrossFormer(seg_len=1, d_model=36, num_classes=2, num_layers=1, win_size=1, n_heads=2, d_ff=16, e_blocks=1)

In [15]:
# model_1

In [16]:
# temp = torch.rand(1,2016, 231)

In [5]:
# torch.cuda.is_available()

In [17]:
# temp.shape

In [18]:
# out = model_1(temp)

In [19]:
# out.shape

In [20]:
# out

### now causal crossformer

In [12]:
from einops import rearrange, repeat
import math


class DSW_embedding(nn.Module):
    def __init__(self, seg_len, d_model):
        super(DSW_embedding, self).__init__()
        self.seg_len = seg_len
        self.d_model = d_model
        self.linear = nn.Linear(seg_len * 231, d_model)

    def forward(self, x):
        batch, ts_len, ts_dim = x.shape
        pad_len = (self.seg_len - (ts_len % self.seg_len)) % self.seg_len
        if pad_len != 0:
            x = torch.cat([x, torch.zeros(batch, pad_len, ts_dim).to(x.device)], dim=1)
        seg_num = (ts_len + pad_len) // self.seg_len
        x_segment = rearrange(x, 'b (seg_num seg_len) d -> b seg_num (seg_len d)', seg_len=self.seg_len)
        x_embed = self.linear(x_segment)
        x_embed = x_embed.unsqueeze(2).repeat(1, 1, self.seg_len, 1)
        x_embed = x_embed.view(batch, -1, self.d_model)
        return x_embed[:, :ts_len, :]


class SegMerging(nn.Module):
    def __init__(self, d_model, win_size, norm_layer=nn.LayerNorm):
        super().__init__()
        self.d_model = d_model
        self.win_size = win_size
        self.linear_trans = nn.Linear(d_model, d_model)
        self.norm = norm_layer(d_model)

    def forward(self, x):
        
        x = self.norm(x)
        x = self.linear_trans(x)
        return x

class AttentionLayer(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = d_model // n_heads

        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        self.o_proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v):
        B, Lq, _ = q.size()
        _, Lk, _ = k.size()

        qkv = self.qkv_proj(q).chunk(3, dim=-1)
        q, k, v = [rearrange(x, 'b l (h d) -> b h l d', h=self.n_heads) for x in qkv]

        scores = torch.einsum('bhqd,bhkd->bhqk', q, k) / math.sqrt(self.d_k)

        if Lq == Lk:
            causal_mask = torch.tril(torch.ones(Lq, Lk, device=q.device)).unsqueeze(0).unsqueeze(0)
            scores = scores.masked_fill(causal_mask == 0, float('-inf'))

        attn = self.dropout(scores.softmax(dim=-1))

        context = torch.einsum('bhqk,bhkd->bhqd', attn, v)
        context = rearrange(context, 'b h l d -> b l (h d)')
        return self.o_proj(context)

class CrossEncoder(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout, factor, e_blocks):
        super().__init__()
        self.time_attention = AttentionLayer(d_model, n_heads, dropout)
        self.dim_sender = AttentionLayer(d_model, n_heads, dropout)
        self.dim_receiver = AttentionLayer(d_model, n_heads, dropout)

        self.router = nn.Parameter(torch.randn(e_blocks, factor, d_model))

        self.dropout = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.norm4 = nn.LayerNorm(d_model)

        self.MLP1 = nn.Sequential(nn.Linear(d_model, d_ff), nn.GELU(), nn.Linear(d_ff, d_model))
        self.MLP2 = nn.Sequential(nn.Linear(d_model, d_ff), nn.GELU(), nn.Linear(d_ff, d_model))

    def forward(self, x):
        time_enc = self.time_attention(x, x, x)
        x = x + self.dropout(time_enc)
        x = self.norm1(x)
        x = x + self.dropout(self.MLP1(x))
        x = self.norm2(x)

        batch_router = repeat(self.router, 'e_blocks factor d_model -> (repeat e_blocks) factor d_model', repeat=x.size(0))

        dim_buffer = self.dim_sender(batch_router, x, x)
        dim_receive = self.dim_receiver(x, dim_buffer, dim_buffer)

        x = x + self.dropout(dim_receive)
        x = self.norm3(x)
        x = x + self.dropout(self.MLP2(x))
        x = self.norm4(x)

        return x

class EncoderOnlyCrossFormer(nn.Module):
    def __init__(self, seg_len, d_model, num_classes, num_layers, n_heads, d_ff, e_blocks, dropout=0.1, win_size=1):
        super().__init__()
        self.embedding = DSW_embedding(seg_len, d_model)
        
        self.encoders = nn.ModuleList([
            CrossEncoder(d_model, n_heads, d_ff, dropout, factor=5, e_blocks=e_blocks)
            for _ in range(num_layers)
        ])
        
        # Include the segment merging layer
        self.seg_merge = SegMerging(d_model, win_size)
        
        # Classification head structure as in the original model
        self.logit = nn.Linear(d_model, num_classes)
        self.classifier = nn.Sequential(
            nn.LayerNorm(d_model),
            self.logit
        )

    def forward(self, x):
        x_embed = self.embedding(x)

        for encoder in self.encoders:
            x_embed = encoder(x_embed)

        # segment merging (required for compatibility)
        x_merged = self.seg_merge(x_embed)

        logits = self.classifier(x_merged)
        return logits


In [13]:
model_1 = EncoderOnlyCrossFormer(seg_len=1, d_model=36, num_classes=2, num_layers=1, n_heads=2, d_ff=16, e_blocks=1)

In [3]:
# model_1

In [14]:
temp = torch.rand(1,2016, 231)

In [15]:
temp.shape

torch.Size([1, 2016, 231])

In [16]:
out = model_1(temp)

In [17]:
out

tensor([[[ 0.0480, -0.6035],
         [ 0.1179, -0.1640],
         [ 0.2247, -0.2564],
         ...,
         [ 0.3353,  0.3213],
         [ 0.6664,  0.1096],
         [ 0.5011, -0.3630]]], grad_fn=<ViewBackward0>)

In [18]:
import warnings

# Suppress warnings from PyTorch
warnings.filterwarnings("ignore", category=UserWarning)
torch.set_printoptions(profile="default")  # Prevent any potential printing-related warnings
import logging
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.accelerators.cuda").setLevel(logging.WARNING)

# Suppress TensorFlow logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Suppress Python warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [19]:
model_type = 'Crossformer'

In [20]:
model_path_1 = 'circ_crossformer/testing_Crossformer_Circ_seed_0/model_epoch_40.pth' #model 1

In [21]:
temp_model = torch.load(model_path_1)
# Extract the model state dictionary
model_state_dict = temp_model['model_state_dict']

In [22]:
model_1.load_state_dict(model_state_dict)

<All keys matched successfully>